In [1]:
from pathlib import Path
import os
def load_env_vars_from_file(path: Path):
    file = path.read_text()
    for line in file.splitlines():
        if line.startswith("export"):
            var_to_value = line.split(" ")[1].split("=")
            var_ = var_to_value[0]
            value_ = var_to_value[1][1:-1]
            os.environ[var_] = value_
def set_up_prod_credentials():
    file = "../../../../scripts/prod_setup_creds.sh"
    load_env_vars_from_file(Path(file))
set_up_prod_credentials()

In [2]:
from exchange.interface import ExchangeInterface

e = ExchangeInterface(is_test_run=False)

In [3]:
from datetime import datetime
from helpers.types.portfolio import GetFillsRequest


fills = e.get_fills(GetFillsRequest(min_ts=int(datetime(2024, 9, 1).timestamp())))

In [5]:
fills[0]

OrderFill(action=<TradeType.SELL: 'sell'>, count=30, created_time=datetime.datetime(2024, 9, 18, 12, 44, 12, 964591, tzinfo=TzInfo(UTC)), is_taker=True, no_price=57, order_id='d404e238-248e-4fe8-a5b3-13c009518c39', side=<Side.NO: 'no'>, ticker='FED-24SEP-T5.00', trade_id='aa06cef0-ecda-4115-b8de-770daa3839d6', yes_price=43)

In [11]:
from collections import defaultdict
from dataclasses import dataclass

from helpers.types.orders import Side, TradeType

@dataclass
class ProfitData():
    positions: int = 0 
    spent: int = 0 
    made: int = 0 
# From market ticker to position
pd = defaultdict(lambda: ProfitData())

for fill in fills:
    if (fill.side == Side.YES and fill.action == TradeType.BUY) or (fill.side == Side.NO and fill.action == TradeType.SELL):
        pd[fill.ticker].spent += (fill.yes_price) * (fill.count)
        if pd[fill.ticker].positions * fill.count < 0:
            pd[fill.ticker].made += fill.count
        pd[fill.ticker].positions += fill.count
    else:
        pd[fill.ticker].spent += (fill.no_price) * (fill.count)
        if pd[fill.ticker].positions * fill.count < 0:
            pd[fill.ticker].made += fill.count
        pd[fill.ticker].positions -= fill.count



In [12]:
pd

defaultdict(<function __main__.<lambda>()>,
            {'FED-24SEP-T5.00': ProfitData(positions=60, spent=4210, made=0),
             'FEDRATEMIN-24DEC31-T4.50': ProfitData(positions=24, spent=670, made=0),
             'HIGHNY-24SEP16-B77.5': ProfitData(positions=30, spent=3490, made=0),
             'HIGHNY-24SEP16-B75.5': ProfitData(positions=10, spent=1474, made=0),
             'HIGHNY-24SEP16-B73.5': ProfitData(positions=-10, spent=890, made=0),
             'HIGHCHI-24SEP15-B89.5': ProfitData(positions=-10, spent=4710, made=40),
             'HIGHNY-24SEP16-B79.5': ProfitData(positions=10, spent=1150, made=0),
             'HIGHAUS-24SEP16-B99.5': ProfitData(positions=-10, spent=1700, made=10),
             'HIGHAUS-24SEP16-B97.5': ProfitData(positions=0, spent=2040, made=0),
             'OSCARNOMACTO-25-TC': ProfitData(positions=-10, spent=440, made=0),
             'RTSPEAKNOEVIL-85': ProfitData(positions=0, spent=3570, made=40),
             'MCSILENTHILL2-84': ProfitData(p